# standardize signal (psc or zscore)

In [ ]:
import os
import glob
import numpy as np
import nibabel as nb
from nipype.interfaces import fsl
import nipype.pipeline.engine as pe
from nilearn.image import resample_to_img
from nilearn import image
from joblib import Parallel, delayed

In [8]:
# setup dirs
home_dir = '/home/shared/2018/visual/cerebellum_prf/'
der_dir = os.path.join(home_dir,'derivatives')
out_dir = os.path.join(home_dir,'derivatives','pp')

# in_dir = os.path.join(out_dir,'ng')
# in_postFix = '_resampled_fnirted_smoothed_sgtf_ng'


in_dir = os.path.join(out_dir,'sgtf')
in_postFix = '_resampled_fnirted_smoothed_sgtf'

subs = ['02','03']#,'02','03']
sess = {
    '01':['01','03','02'],
    '02':['01','02','03','04'],
    '03':['01','02','03']
}

TR = 1.5
space = 'MNI152NLin2009cAsym' # 'T1w'

dtype = 'zscore' # or psc
stand_out_dir = os.path.join(out_dir,dtype)
if not os.path.isdir(stand_out_dir): os.mkdir(stand_out_dir)


In [9]:
def perform_standardization(sj_sgtf_out_dir,sj_stand_out_dir,fn,dtype):
    
    out_fn = fn.replace(sj_sgtf_out_dir,sj_stand_out_dir).replace('.nii.gz','_%s.nii.gz'%dtype)

    if not os.path.isfile(out_fn):

        # load img
        img = nb.load(fn)
        data = img.get_data()

        mdata = np.nanmedian(data,axis=-1)[:,:,:,np.newaxis]
        stddata = np.nanstd(data,axis=-1)[:,:,:,np.newaxis]

        if dtype == 'zscore':
            standard_data = (data - mdata) / stddata
        elif dtype == 'psc':
            standard_data =  100.0 * (data - mdata) / mdata

        new_img = nb.Nifti1Image(standard_data,affine=img.affine,header=img.header)

        nb.save(new_img,out_fn)    

In [10]:
for sub in subs:
    for ses in sess[sub]:
        print('now computing %s for sub %s, ses %s'%(dtype,sub,ses))

        sj_out_dir = os.path.join(stand_out_dir,'sub-%s'%sub)
        if not os.path.isdir(sj_out_dir): os.mkdir(sj_out_dir)

        sj_in_dir = os.path.join(in_dir,'sub-%s'%sub)
        
        fns = sorted(glob.glob(os.path.join(sj_in_dir,'sub-%s_ses-%s*bold_space-%s_preproc%s.nii.gz'%(sub,ses,space,in_postFix))))
        
        Parallel(n_jobs=6,verbose=9)(delayed(perform_standardization)(sj_in_dir,sj_out_dir,fn,dtype)  for fn in fns)

print 'done!'

now computing zscore for sub 02, ses 01


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   4 out of  13 | elapsed:  1.0min remaining:  2.3min
[Parallel(n_jobs=6)]: Done   6 out of  13 | elapsed:  1.1min remaining:  1.2min
[Parallel(n_jobs=6)]: Done   8 out of  13 | elapsed:  2.1min remaining:  1.3min
[Parallel(n_jobs=6)]: Done  10 out of  13 | elapsed:  2.1min remaining:   38.4s
[Parallel(n_jobs=6)]: Done  13 out of  13 | elapsed:  3.0min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


now computing zscore for sub 02, ses 02


[Parallel(n_jobs=6)]: Done   2 out of   8 | elapsed:  1.1min remaining:  3.2min
[Parallel(n_jobs=6)]: Done   3 out of   8 | elapsed:  1.1min remaining:  1.8min
[Parallel(n_jobs=6)]: Done   4 out of   8 | elapsed:  1.1min remaining:  1.1min
[Parallel(n_jobs=6)]: Done   5 out of   8 | elapsed:  1.1min remaining:   38.8s
[Parallel(n_jobs=6)]: Done   6 out of   8 | elapsed:  1.1min remaining:   22.6s
[Parallel(n_jobs=6)]: Done   8 out of   8 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=6)]: Done   8 out of   8 | elapsed:  2.1min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


now computing zscore for sub 02, ses 03


...

[Parallel(n_jobs=6)]: Done   3 out of  12 | elapsed:   57.2s remaining:  2.9min
[Parallel(n_jobs=6)]: Done   5 out of  12 | elapsed:   57.9s remaining:  1.4min
[Parallel(n_jobs=6)]: Done   7 out of  12 | elapsed:  1.9min remaining:  1.3min
[Parallel(n_jobs=6)]: Done   9 out of  12 | elapsed:  1.9min remaining:   37.8s
[Parallel(n_jobs=6)]: Done  12 out of  12 | elapsed:  1.9min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


now computing zscore for sub 03, ses 03


[Parallel(n_jobs=6)]: Done   3 out of  12 | elapsed:  8.5min remaining: 25.5min
[Parallel(n_jobs=6)]: Done   5 out of  12 | elapsed:  8.5min remaining: 11.9min
[Parallel(n_jobs=6)]: Done   7 out of  12 | elapsed:  9.6min remaining:  6.9min
[Parallel(n_jobs=6)]: Done   9 out of  12 | elapsed:  9.6min remaining:  3.2min


done!


[Parallel(n_jobs=6)]: Done  12 out of  12 | elapsed:  9.6min finished
